# XGBoost

In [ ]:
# https://www.kaggle.com/c/titanic

In [180]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score , roc_auc_score

In [181]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [182]:
# Загружаем данные из файлов
X_train = pd.read_csv('./titanic/titanic_cleaned_train.csv')
y_train = pd.read_csv('./titanic/titanic_cleaned_train_y.csv', header=None)
X_test = pd.read_csv('./titanic/titanic_cleaned_test.csv')

In [183]:
test = pd.read_csv('titanic/test.csv')

In [184]:
X_train.head()

,SibSp,Parch,Has_Cabin,IsAlone,Pclass_1,Pclass_2,Pclass_3,Age_0.0,Age_1.0,Age_2.0,...,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11,clusters_0,clusters_1
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [185]:
for_drop = ["clusters_0", 
            "clusters_1" ,
            "IsAlone" ,
            "Age_4.0", 
            "FamilySize_7", 
            "FamilySize_8",
            "FamilySize_11",
            "Pclass_2",
            "FamilySize_2", 
            "Title_3",
            "FamilySize_3",
            "Title_2",
            "Title_4",
            "FamilySize_1",
            "FamilySize_4",]
for_drop = []
X_train.drop(for_drop, inplace=True, axis=1) #"clusters_0",  "Parch", "Miss","Ms", "Mrs"
X_test.drop(for_drop, inplace=True, axis=1)

## Шаг 1: Зафиксируем learning_rate и параметры дерева и подберём n_estimators

Параметры:

* **max_depth**. Как указанов в таблице выше, обычно варьируется в интервале от 3 до 10 (но от задачи к задаче значения могут меняться). В качестве начального значения обычно используют 5
* **min_child_weight**. Если выборка сильно несбалансирована, то лучше выбрать значение "1". Иначе лучше выбрать значение "2" и зафиксировать
* **gamma**. Обычно выставляют значение в интервале от 0 до 0.2 и фиксируют. В дальнейшем этот параметр всегда можно затюнить отдельно
* **subsample, colsample_bytree**. Выставим 0.8 и зафиксируем. Можно также проварьировать в интервале 0.5-0.9.
* **scale_pos_weight**. Выставляется в зафисимости от соотношения классов в выборке и фиксируется

In [186]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

ones_ratio = y_train[y_train == 1].shape[0] * 1.0 / y_train[y_train == 0].shape[0] # посчитаем соотношение между классами

param_grid = {
    # параметры ансамбля
    'n_estimators': list(np.arange(10, 1000, 50)),#[10, 30, 50, 100, 200, 400, 600, 1000],
    'learning_rate': [0.01],
    
    # параметры дерева
    'max_depth': [5],
    'min_child_weight': [2],
    'gamma': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [ones_ratio],
    
    # параметры регуляризации
    'reg_alpha': [0.0],
    'reg_lambda': [1.0]
}

cv = KFold(n_splits=4, shuffle=True)

clf = xgboost.XGBClassifier()
gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.87149426617
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 2,
 'missing': None,
 'n_estimators': 260,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 2. Подбираем параметры дерева
* **max_depth** - будем варьировать от 3 до 10 с шагом 2
* **min_child_weight** - от 1 до 6 с шагом 2

In [198]:
param_grid = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}

clf = xgboost.XGBClassifier(**best_params) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.886448872502
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1.0,
 'gamma': 0.2,
 'learning_rate': 0.00392156862745098,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 510,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'reg_lambda': 1,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 0.9}

## Шаг 3. Подбираем gamma (критерий создания поддерева)
* **gamma** - от 0 до 0.5 с шагом 0.1

In [199]:
param_grid = {
    'gamma': [0.1*i for i in range(12)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.877153314647
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1.0,
 'gamma': 0.1,
 'learning_rate': 0.00392156862745098,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 510,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'reg_lambda': 1,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 0.9}

## Шаг 4. Затюним subsample и colsample_bytree
* **subsample** - от 0.5 до 1.0 с шагом 0.1
* **colsample_bytree** - от 0.5 до 1.0 с шагом 0.1

In [200]:
param_grid = {
    'subsample': [0.5 + 0.1*i for i in range(6)],
    'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.878848928102
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.1,
 'learning_rate': 0.00392156862745098,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 510,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'reg_lambda': 1,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 5. Регуляризация
* **reg_alpha** [1e-5, 1e-2, 0.1, 1, 100]
* **reg_lambda** [1e-5, 1e-2, 0.1, 1, 100]

In [202]:
param_grid = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 0.2, 0.3, 0.5, 1, 2, 3],
    'reg_lambda': [1e-5, 1e-2, 0.1, 0.2, 0.3, 0.5, 1, 2, 3]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.875423850025
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.1,
 'learning_rate': 0.00392156862745098,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 510,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.2,
 'reg_lambda': 0.3,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 6. Learning rate
Чем меньше у нас **n_estimators** в ансамбле, тем быстрее нам нужно двигаться с каждым шагом (добавлением нового классификатора), т.е. делать больший **learning_rate**. Обычно **learning rate** варьируют так, чтобы произведение **n_estimators** x **learning_rate** оставалось инвариантным

In [203]:
import numpy as np


clf = xgboost.XGBClassifier(**best_params)
best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
best_learning_rate = best_params['learning_rate'] # аналогичная запись
invariant_composition = best_n_estimators * best_learning_rate
n_estimators_range = np.arange(10, 1000, 50)#[10, 30, 100, 200, 400, 600, 800, 1000]

best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

for n_estimators in n_estimators_range:
    learning_rate = invariant_composition / n_estimators
    clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
    aucs = []
    for train_idx, test_idx in cv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold, verbose=0)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
    auc = np.mean(auc)
    if auc > best_score:
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_score = auc
        
best_params['n_estimators'] = best_n_estimators
best_params['learning_rate'] = best_learning_rate

print('Best score (AUC): ', best_score)
print('Best params: ')
best_params

Best score (AUC):  0.935079225352
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.1,
 'learning_rate': 0.0076923076923076927,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 260,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.2,
 'reg_lambda': 0.3,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

# Выведем важность признаков

In [204]:
import operator
z = sorted(zip(X_train.columns, clf.feature_importances_), key=operator.itemgetter(1), reverse=True)

for feature_name, feature_importance in z:
    print('Feature: "%s"\tFeature importance: %.4f' % (feature_name, feature_importance))

Feature: "Fare_3"	Feature importance: 0.0942
Feature: "Has_Cabin"	Feature importance: 0.0746
Feature: "SibSp"	Feature importance: 0.0743
Feature: "Pclass_3"	Feature importance: 0.0698
Feature: "IsAlone"	Feature importance: 0.0629
Feature: "Pclass_1"	Feature importance: 0.0579
Feature: "Title_1"	Feature importance: 0.0504
Feature: "Fare_0"	Feature importance: 0.0482
Feature: "Age_1.0"	Feature importance: 0.0478
Feature: "Title_5"	Feature importance: 0.0438
Feature: "Fare_1"	Feature importance: 0.0400
Feature: "FamilySize_7"	Feature importance: 0.0362
Feature: "Age_3.0"	Feature importance: 0.0362
Feature: "FamilySize_4"	Feature importance: 0.0357
Feature: "Title_2"	Feature importance: 0.0346
Feature: "Fare_2"	Feature importance: 0.0344
Feature: "Age_0.0"	Feature importance: 0.0296
Feature: "Parch"	Feature importance: 0.0185
Feature: "Title_4"	Feature importance: 0.0182
Feature: "FamilySize_2"	Feature importance: 0.0170
Feature: "Age_2.0"	Feature importance: 0.0167
Feature: "FamilySize_6"

# Обучим модель с лучшими параметрами и отправим на kaggle

In [208]:
best_params = {'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.1,
 'learning_rate': 0.0076923076923076927,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 260,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.2,
 'reg_lambda': 0.3,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

clf = xgboost.XGBClassifier(**best_params)


In [209]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.1, learning_rate=0.007692307692307693, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=260,
       nthread=-1, objective='binary:logistic', reg_alpha=0.2,
       reg_lambda=0.3, scale_pos_weight=1.0, seed=0, silent=True,
       subsample=1.0)

In [210]:
predictions = clf.predict(X_test)
predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [196]:
submussion = 'PassengerId,Survived\n'
submussion += "\n".join(["{},{}".format(pid, prediction) for pid, prediction in zip(test.PassengerId, predictions)])

In [197]:
with open('submission.txt', 'w') as file:
    file.write(submussion)